# LSTM Encoder Decoder

<b>reference</b></br>
https://github.com/KDD-OpenSource/DeepADoTS<br/>
https://stackoverflow.com/questions/42415909/initializing-lstm-hidden-state-tensorflow-keras

In [109]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [110]:
# data probing
scaler = MinMaxScaler()
df = pd.read_csv('jena_climate_2009_2016.csv').head(1029)
df_train = df.drop(columns=['Date Time'])
df_train[df_train.columns.values] = scaler.fit_transform(df_train[df_train.columns.values])
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1029 entries, 0 to 1028
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   p (mbar)         1029 non-null   float64
 1   T (degC)         1029 non-null   float64
 2   Tpot (K)         1029 non-null   float64
 3   Tdew (degC)      1029 non-null   float64
 4   rh (%)           1029 non-null   float64
 5   VPmax (mbar)     1029 non-null   float64
 6   VPact (mbar)     1029 non-null   float64
 7   VPdef (mbar)     1029 non-null   float64
 8   sh (g/kg)        1029 non-null   float64
 9   H2OC (mmol/mol)  1029 non-null   float64
 10  rho (g/m**3)     1029 non-null   float64
 11  wv (m/s)         1029 non-null   float64
 12  max. wv (m/s)    1029 non-null   float64
 13  wd (deg)         1029 non-null   float64
dtypes: float64(14)
memory usage: 112.7 KB


In [113]:
# * 探索代码
batch_size = 20
TIME_STEPS = 30

# Generated training sequences for use in the model.
def create_sequences(values, time_steps=TIME_STEPS):
    output = []
    for i in range(len(values) - time_steps + 1):
        output.append(values[i : (i + time_steps)])
    return np.stack(output)

x_train = create_sequences(df_train.values)
print("Training input shape: ", x_train.shape)

inputs = Input(shape=(x_train.shape[1], x_train.shape[2]))
outputs = LSTMED(units=64, batch_size=batch_size, training=True)(inputs)

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="mse")
model.summary()

Training input shape:  (1000, 30, 14)
Model: "model_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_45 (InputLayer)       [(None, 30, 14)]          0         
                                                                 
 lstmed_42 (LSTMED)          (20, 30, 14)              41358     
                                                                 
Total params: 41,358
Trainable params: 41,358
Non-trainable params: 0
_________________________________________________________________


In [114]:
'''
    tensorflow使用静态图模型，对lstm encoder decoder的支持存在下面的问题：

    需要满足 训练数据集行数(1029) - TIME_STEPS + 1 能够整除 batch_size / validation_split
    考虑使用 Sequence 接口解决这个问题.
'''
history = model.fit(
    x_train,
    x_train,
    epochs=10,
    batch_size=batch_size,
    validation_split=0.1
)

Epoch 1/10
45/45 [==============================] - 45s 255ms/step - loss: 0.0942 - val_loss: 0.0200
Epoch 2/10
45/45 [==============================] - 1s 31ms/step - loss: 0.0196 - val_loss: 0.0086
Epoch 3/10
45/45 [==============================] - 1s 30ms/step - loss: 0.0117 - val_loss: 0.0063
Epoch 4/10
45/45 [==============================] - 1s 28ms/step - loss: 0.0081 - val_loss: 0.0057
Epoch 5/10
45/45 [==============================] - 1s 27ms/step - loss: 0.0059 - val_loss: 0.0047
Epoch 6/10
45/45 [==============================] - 1s 28ms/step - loss: 0.0050 - val_loss: 0.0042
Epoch 7/10
45/45 [==============================] - 1s 27ms/step - loss: 0.0047 - val_loss: 0.0041
Epoch 8/10
45/45 [==============================] - 1s 28ms/step - loss: 0.0045 - val_loss: 0.0039
Epoch 9/10
45/45 [==============================] - 1s 28ms/step - loss: 0.0044 - val_loss: 0.0037
Epoch 10/10
45/45 [==============================] - 1s 27ms/step - loss: 0.0043 - val_loss: 0.0034


# Code Repo

TensorFlow版本

In [98]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, LSTM, Layer, Input
from tensorflow.keras import Model

class LSTMED(Layer):
    '''
        @param units number of hidden units
    '''
    def __init__(self, units, batch_size, training=False):
        super(LSTMED, self).__init__()
        self.units = units
        self.batch_size = batch_size
        self.training = training
    
    def build(self, input_shape):
        time_steps = input_shape[1]
        n_features = input_shape[-1]
        # multi step lstm
        self.encoder = LSTM(self.units, return_state=True, name="encoder", kernel_initializer='Zeros', recurrent_initializer='Zeros', input_shape=(time_steps, n_features))
        # single step lstm
        self.decoder = LSTM(self.units, return_state=True, name="decoder", kernel_initializer='Zeros', recurrent_initializer='Zeros', input_shape=(1, n_features))
        self.hidden2output = Dense(n_features)
    
    def call(self, inputs):
        batch_size = self.batch_size
        h_0 = tf.zeros(shape=(batch_size, self.units))
        c_0 = tf.zeros(shape=(batch_size, self.units))
        # 1. Encode the timeseries to make use of the last hidden state.
        _, h_0, c_0 = self.encoder(inputs, initial_state=[h_0, c_0])  # .float() here or .double() for the model

        # 2. Use hidden state as initialization for our Decoder-LSTM

        # 3. Also, use this hidden state to get the first output aka the last point of the reconstructed timeseries
        # 4. Reconstruct timeseries backwards
        #    * Use true data for training decoder
        #    * Use hidden2output for prediction
        output = tf.unstack(tf.transpose(tf.zeros_like(inputs), perm=[1,0,2]))
        for i in reversed(range(inputs.shape[1])):
            output[i] = self.hidden2output(h_0)

            if self.training:
                _, h_0, c_0 = self.decoder(tf.expand_dims(inputs[:, i], axis=1), initial_state=[h_0, c_0])
            else:
                _, h_0, c_0 = self.decoder(tf.expand_dims(output[:, i], axis=1), initial_state=[h_0, c_0])

        return tf.transpose(tf.stack(output), perm=[1,0,2])

PyTorch版本

In [ ]:
# 启动代码
df_train = pd.DataFrame(df.iloc[:, 4].values.reshape(-1, 1))
model = LSTMED()
model.fit(df_train)

In [33]:
import abc
import logging
import random

import numpy as np
import torch
from torch.autograd import Variable

import pandas as pd
import torch
import torch.nn as nn
from scipy.stats import multivariate_normal
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from tqdm import trange

class Algorithm(metaclass=abc.ABCMeta):
    def __init__(self, module_name, name, seed, details=False):
        self.logger = logging.getLogger(module_name)
        self.name = name
        self.seed = seed
        self.details = details
        self.prediction_details = {}

        if self.seed is not None:
            random.seed(seed)
            np.random.seed(seed)

    def __str__(self):
        return self.name

    @abc.abstractmethod
    def fit(self, X):
        """
        Train the algorithm on the given dataset
        """

    @abc.abstractmethod
    def predict(self, X):
        """
        :return anomaly score
        """


class PyTorchUtils(metaclass=abc.ABCMeta):
    def __init__(self, seed, gpu):
        self.gpu = gpu
        self.seed = seed
        if self.seed is not None:
            torch.manual_seed(self.seed)
            torch.cuda.manual_seed(self.seed)
        self.framework = 0

    @property
    def device(self):
        return torch.device(f'cuda:{self.gpu}' if torch.cuda.is_available() and self.gpu is not None else 'cpu')

    def to_var(self, t, **kwargs):
        # ToDo: check whether cuda Variable.
        t = t.to(self.device)
        return Variable(t, **kwargs)

    def to_device(self, model):
        model.to(self.device)

class LSTMED(Algorithm, PyTorchUtils):
    def __init__(self, name: str = 'LSTM-ED', num_epochs: int = 10, batch_size: int = 20, lr: float = 1e-3,
                 hidden_size: int = 5, sequence_length: int = 30, train_gaussian_percentage: float = 0.25,
                 n_layers: tuple = (1, 1), use_bias: tuple = (True, True), dropout: tuple = (0, 0),
                 seed: int = None, gpu: int = None, details=True):
        Algorithm.__init__(self, __name__, name, seed, details=details)
        PyTorchUtils.__init__(self, seed, gpu)
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.lr = lr

        self.hidden_size = hidden_size
        self.sequence_length = sequence_length
        self.train_gaussian_percentage = train_gaussian_percentage

        self.n_layers = n_layers
        self.use_bias = use_bias
        self.dropout = dropout

        self.lstmed = None
        self.mean, self.cov = None, None

    def fit(self, X: pd.DataFrame):
        X.interpolate(inplace=True)
        X.bfill(inplace=True)
        data = X.values
        sequences = [data[i:i + self.sequence_length] for i in range(data.shape[0] - self.sequence_length + 1)]
        indices = np.random.permutation(len(sequences))
        split_point = int(self.train_gaussian_percentage * len(sequences))
        train_loader = DataLoader(dataset=sequences, batch_size=self.batch_size, drop_last=True,
                                  sampler=SubsetRandomSampler(indices[:-split_point]), pin_memory=True)
        train_gaussian_loader = DataLoader(dataset=sequences, batch_size=self.batch_size, drop_last=True,
                                           sampler=SubsetRandomSampler(indices[-split_point:]), pin_memory=True)

        self.lstmed = LSTMEDModule(X.shape[1], self.hidden_size,
                                   self.n_layers, self.use_bias, self.dropout,
                                   seed=self.seed, gpu=self.gpu)
        self.to_device(self.lstmed)
        optimizer = torch.optim.Adam(self.lstmed.parameters(), lr=self.lr)

        self.lstmed.train()
        for epoch in trange(self.num_epochs):
            logging.debug(f'Epoch {epoch+1}/{self.num_epochs}.')
            for ts_batch in train_loader:
                output = self.lstmed(self.to_var(ts_batch))
                loss = nn.MSELoss(size_average=False)(output, self.to_var(ts_batch.float()))
                self.lstmed.zero_grad()
                loss.backward()
                optimizer.step()

        self.lstmed.eval()
        error_vectors = []
        for ts_batch in train_gaussian_loader:
            output = self.lstmed(self.to_var(ts_batch))
            error = nn.L1Loss(reduce=False)(output, self.to_var(ts_batch.float()))
            error_vectors += list(error.view(-1, X.shape[1]).data.cpu().numpy())

        self.mean = np.mean(error_vectors, axis=0)
        self.cov = np.cov(error_vectors, rowvar=False)

    def predict(self, X: pd.DataFrame):
        X.interpolate(inplace=True)
        X.bfill(inplace=True)
        data = X.values
        sequences = [data[i:i + self.sequence_length] for i in range(data.shape[0] - self.sequence_length + 1)]
        data_loader = DataLoader(dataset=sequences, batch_size=self.batch_size, shuffle=False, drop_last=False)

        self.lstmed.eval()
        mvnormal = multivariate_normal(self.mean, self.cov, allow_singular=True)
        scores = []
        outputs = []
        errors = []
        for idx, ts in enumerate(data_loader):
            output = self.lstmed(self.to_var(ts))
            error = nn.L1Loss(reduce=False)(output, self.to_var(ts.float()))
            score = -mvnormal.logpdf(error.view(-1, X.shape[1]).data.cpu().numpy())
            scores.append(score.reshape(ts.size(0), self.sequence_length))
            if self.details:
                outputs.append(output.data.numpy())
                errors.append(error.data.numpy())

        # stores seq_len-many scores per timestamp and averages them
        scores = np.concatenate(scores)
        lattice = np.full((self.sequence_length, data.shape[0]), np.nan)
        for i, score in enumerate(scores):
            lattice[i % self.sequence_length, i:i + self.sequence_length] = score
        scores = np.nanmean(lattice, axis=0)

        if self.details:
            outputs = np.concatenate(outputs)
            lattice = np.full((self.sequence_length, X.shape[0], X.shape[1]), np.nan)
            for i, output in enumerate(outputs):
                lattice[i % self.sequence_length, i:i + self.sequence_length, :] = output
            self.prediction_details.update({'reconstructions_mean': np.nanmean(lattice, axis=0).T})

            errors = np.concatenate(errors)
            lattice = np.full((self.sequence_length, X.shape[0], X.shape[1]), np.nan)
            for i, error in enumerate(errors):
                lattice[i % self.sequence_length, i:i + self.sequence_length, :] = error
            self.prediction_details.update({'errors_mean': np.nanmean(lattice, axis=0).T})

        return scores


class LSTMEDModule(nn.Module, PyTorchUtils):
    def __init__(self, n_features: int, hidden_size: int,
                 n_layers: tuple, use_bias: tuple, dropout: tuple,
                 seed: int, gpu: int):
        super().__init__()
        PyTorchUtils.__init__(self, seed, gpu)
        self.n_features = n_features
        self.hidden_size = hidden_size

        self.n_layers = n_layers
        self.use_bias = use_bias
        self.dropout = dropout

        self.encoder = nn.LSTM(self.n_features, self.hidden_size, batch_first=True,
                               num_layers=self.n_layers[0], bias=self.use_bias[0], dropout=self.dropout[0])
        self.to_device(self.encoder)
        self.decoder = nn.LSTM(self.n_features, self.hidden_size, batch_first=True,
                               num_layers=self.n_layers[1], bias=self.use_bias[1], dropout=self.dropout[1])
        self.to_device(self.decoder)
        self.hidden2output = nn.Linear(self.hidden_size, self.n_features)
        self.to_device(self.hidden2output)

    def _init_hidden(self, batch_size):
        return (self.to_var(torch.Tensor(self.n_layers[0], batch_size, self.hidden_size).zero_()),
                self.to_var(torch.Tensor(self.n_layers[0], batch_size, self.hidden_size).zero_()))

    def forward(self, ts_batch, return_latent: bool = False):
        batch_size = ts_batch.shape[0]

        # 1. Encode the timeseries to make use of the last hidden state.
        enc_hidden = self._init_hidden(batch_size)  # initialization with zero
        _, enc_hidden = self.encoder(ts_batch.float(), enc_hidden)  # .float() here or .double() for the model

        # 2. Use hidden state as initialization for our Decoder-LSTM
        dec_hidden = enc_hidden

        # 3. Also, use this hidden state to get the first output aka the last point of the reconstructed timeseries
        # 4. Reconstruct timeseries backwards
        #    * Use true data for training decoder
        #    * Use hidden2output for prediction
        output = self.to_var(torch.Tensor(ts_batch.size()).zero_())
        for i in reversed(range(ts_batch.shape[1])):
            output[:, i, :] = self.hidden2output(dec_hidden[0][0, :])

            if self.training:
                _, dec_hidden = self.decoder(ts_batch[:, i].unsqueeze(1).float(), dec_hidden)
            else:
                _, dec_hidden = self.decoder(output[:, i].unsqueeze(1), dec_hidden)

        return (output, enc_hidden[1][-1]) if return_latent else output